# Example IMD Radar Data

In [ ]:
import os
import glob
import pyart
import requests
import subprocess
import pyscancf as pcf

In [ ]:
# GitHub raw content base URL (for downloading)
base_url = (
    "https://raw.githubusercontent.com/syedhamidali/pyscancf_examples/main/data/goa16/"
)
api_url = (
    "https://api.github.com/repos/syedhamidali/pyscancf_examples/contents/data/goa16"
)
response = requests.get(api_url)
files = [item["name"] for item in response.json()]
print(files)

In [ ]:
# Target directory to save downloaded files
target_dir = os.path.join(os.getenv("GITHUB_WORKSPACE", "."), "goa_data")
os.makedirs(target_dir, exist_ok=True)


# Function to download files using curl
def download_with_curl(file_name):
    file_url = base_url + file_name
    file_path = os.path.join(target_dir, file_name)

    if not os.path.exists(file_path):
        print(f"Downloading {file_name}...")
        subprocess.run(["curl", "-o", file_path, file_url], check=True)
        print(f"Downloaded {file_name}")
    else:
        print(f"{file_name} already exists.")

    return file_path


# Download the files
downloaded_files = []
for file_name in files:
    file_path = download_with_curl(file_name)
    downloaded_files.append(file_path)

In [ ]:
input_dir = target_dir
out_dir = "outdir"
!rm -rf $out_dir

In [ ]:
pcf.cfrad(
    input_dir,
    out_dir,
)

In [ ]:
files = sorted(glob.glob(out_dir + "/*.nc"))
files

In [ ]:
for file in files:
    radar = pyart.io.read(file)
    grid = pcf.get_grid(radar, grid_shape=(20, 401, 401), height=10, length=250)
    grid.fields["REF"] = grid.fields.pop("Z")
    pcf.plot_cappi(grid, "REF")